<a href="https://colab.research.google.com/github/HGM0223/2025AI_Giraffe/blob/main/Human_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 내가 기린 그린 기림

1. colab 환경 준비
2. 파이썬, pytorch 다운 그레이드
3. 필수 패키지 설치
4. styleGAN3 코드 다운
5. 데이터셋 준비 (리사이주 zip압축)
6. 학습
7. 이미지 생성

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pwd

'/content'

In [3]:
cd /content/drive/MyDrive/2025AI_Giraffe

/content/drive/MyDrive/2025AI_Giraffe


In [4]:
# 버전 확인 (현재 최신버전으로 import되어 있는데 styleGAN에서 권장하는 버전으로
# 다운그레이드 해야 플러그인 오류가 나지 않음. 아래가 다운그레이드 과정 )
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.6.0+cu124
0.21.0+cu124


In [5]:
# ✅ Python 3.10 설치 (torchvision 0.14.1를 지원하는 python 버전을 설치하는 과정 )
!sudo apt-get update
!sudo apt-get install python3.10 python3.10-distutils python3.10-dev -y

# ✅ Python 3.10을 기본 python3으로 설정
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3

# select 나오면 1 입력 (python3.10 )

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,765 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,017 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

In [6]:
# ✅ get-pip.py를 정확한 주소에서 다운로드
!curl -O https://bootstrap.pypa.io/get-pip.py

# ✅ Python 3.10으로 pip 설치
!python3 get-pip.py

# ✅ pip 업그레이드 (선택)
!python3 -m pip install --upgrade pip setuptools

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2225k  100 2225k    0     0  2614k      0 --:--:-- --:--:-- --:--:-- 2612k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [7]:
# ✅ PyTorch 1.13.1 + torchvision 0.14.1 (CUDA 11.6)
!pip install torch==1.13.1 torchvision==0.14.1 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 169.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [torchvision]


In [8]:
# 필요한 라이브러리 설치
!pip install ninja imageio-ffmpeg
!pip install click requests tqdm pyspng

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [imageio-ffmpeg]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [click]


In [9]:
!git clone https://github.com/NVlabs/stylegan3.git

fatal: destination path 'stylegan3' already exists and is not an empty directory.


In [10]:
%cd /content/drive/MyDrive/2025AI_Giraffe/stylegan3

/content/drive/MyDrive/2025AI_Giraffe/stylegan3


In [11]:
# 사람 데이터셋 업로드용
!git clone https://github.com/HGM0223/2025AI_Giraffe

Cloning into '2025AI_Giraffe'...
remote: Enumerating objects: 2468, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (296/296), done.
remote: Total 2468 (delta 24), reused 226 (delta 8), pack-reused 2164 (from 1)
Receiving objects: 100% (2468/2468), 1.73 GiB | 16.35 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Updating files: 100% (1489/1489), done.
